## EE 461P: Data Science Principles  
### Term Project
### Blog Due: May 13, 2021 
### Presentation May 4, 2021





In [5]:
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SequentialFeatureSelector
import os, sys, re
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics import plot_roc_curve 
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score
from sklearn.naive_bayes import BernoulliNB

#Load dataset from csv file
df = pd.read_csv("PS_20174392719_1491204439457_log.csv")
print("data loaded")

data loaded


In [6]:
#Data preprocessing 
for col in ["type","nameOrig","nameDest"]:
    df[col] = df[col].astype('category')
    df[col] = df[col].cat.codes
df["unequalChangeAccounts"] = np.where((df["oldbalanceOrg"]-df["newbalanceOrig"])!=(df["newbalanceDest"]-df["oldbalanceDest"]),1,0)
df["flowDirection"] = np.where((df["oldbalanceOrg"]-df["newbalanceOrig"])<(df["oldbalanceDest"]-df["newbalanceDest"]),1,0)
df["unequalChangeAmount1"] = np.where((df["amount"])!=abs(df["newbalanceOrig"]-df["oldbalanceOrg"]),1,0)
df["unequalChangeAmount2"] = np.where((df["amount"])!=abs(df["newbalanceDest"]-df["oldbalanceDest"]),1,0)
df = df.drop(["oldbalanceOrg","newbalanceOrig","newbalanceDest","oldbalanceDest","amount","isFlaggedFraud","step"],axis=1)
fraud_rows = df[df['isFraud']==1].copy()
not_fraud_rows = df[df['isFraud']==0].copy()
X=df
y = df.pop("isFraud")
X_tr, X_te, y_tr, y_te = train_test_split(X,y, test_size=0.25, random_state = 42)
f_y = fraud_rows.pop("isFraud")
f_x = fraud_rows
nf_y = not_fraud_rows.pop("isFraud")
nf_x = not_fraud_rows
print("data processed")

data processed


In [14]:
bnb = BernoulliNB(fit_prior=False)
bnb.fit(X_tr,y_tr)
print("Model Accuracy on All Test Data: {}".format(bnb.score(X_te,y_te)))
print("Model Accuracy for predicting true fraud: {}".format(bnb.score(f_x,f_y)))
print("Model Accuracy for predicting true not fraud: {}".format(bnb.score(nf_x,nf_y)))
print(sum(bnb.predict(X)))
print(sum(y))

Model Accuracy on All Test Data: 0.9323832006311865
Model Accuracy for predicting true fraud: 0.9902593449409473
Model Accuracy for predicting true not fraud: 0.932223258598324
438814
8213


In [18]:
print(df.head())
for feature,coef in zip(bnb.classes_,bnb.coef_):
    print(coef)

   type  nameOrig  nameDest  unequalChangeAccounts  flowDirection  \
0     3    757869   1662094                      1              0   
1     3   2188998   1733924                      1              0   
2     4   1002156    439685                      1              0   
3     1   5828262    391696                      1              1   
4     3   3445981    828919                      1              0   

   unequalChangeAmount1  unequalChangeAmount2  
0                     1                     1  
1                     1                     1  
2                     0                     1  
3                     0                     1  
4                     0                     1  
[-1.62087690e-04 -1.62087690e-04 -1.62087690e-04 -3.05671110e-01
 -6.32955884e+00 -4.77621040e+00 -3.05451112e-01]
